# 🧠 Dendritic YOLOv8: Clean L4 GPU Version

**FRESH START** - No kernel issues, optimized for L4 GPU

## Quick Setup for L4 GPU:
1. **Runtime → Change runtime type → L4 GPU → Save**
2. **Runtime → Restart runtime** 
3. **Run cells in order** (each cell is designed to work independently)

In [ ]:
# Set W&B API Key FIRST
import os
os.environ["WANDB_API_KEY"] = "21942b7ed5b0ebedb98e928635acff2e972a99fc"
print("✅ W&B API key set")

In [ ]:
# CLEAN dependency install - no hanging
import sys
print(f"Python: {sys.version}")

# Check if in Colab
IN_COLAB = 'google.colab' in sys.modules
print(f"In Colab: {IN_COLAB}")

if IN_COLAB:
    print("Installing for Colab L4 GPU...")
    !pip install -q ultralytics wandb matplotlib pandas seaborn
    !pip install -q perforatedai==3.0.7
    print("✅ Dependencies installed!")
else:
    print("Local environment - ensure packages are installed")

In [ ]:
# Import and GPU check
import torch
import json
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ultralytics import YOLO
import wandb

# GPU verification
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    device = 'cuda'
    gpu_name = torch.cuda.get_device_name(0)
    memory_gb = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"✅ GPU: {gpu_name}")
    print(f"   Memory: {memory_gb:.1f} GB")
    if 'L4' in gpu_name:
        print("🎯 L4 GPU detected - perfect!")
else:
    device = 'cpu'
    print("⚠️ No GPU - select L4 GPU in Runtime settings")

print(f"Device: {device}")

In [ ]:
# PerforatedAI import with fallback
try:
    from perforatedai import globals_perforatedai as GPA
    from perforatedai import utils_perforatedai as UPA
    PERFORATED_AI_AVAILABLE = True
    print("✅ PerforatedAI imported successfully!")
except ImportError as e:
    print(f"⚠️ PerforatedAI not available: {e}")
    PERFORATED_AI_AVAILABLE = False
    
    # Dummy classes
    class DummyGPA:
        class pc:
            @staticmethod
            def set_testing_dendrite_capacity(val): pass
            @staticmethod
            def set_verbose(val): pass
            @staticmethod
            def set_dendrite_update_mode(val): pass
        class pai_tracker:
            @staticmethod
            def set_optimizer(opt): pass
            @staticmethod
            def set_scheduler(sched): pass
            @staticmethod
            def setup_optimizer(model, opt_args, sched_args): 
                import torch.optim as optim
                return optim.Adam(model.parameters(), **opt_args), None
            @staticmethod
            def add_validation_score(score, model):
                return model, False, True
    
    class DummyUPA:
        @staticmethod
        def initialize_pai(model, **kwargs):
            return model
    
    GPA = DummyGPA()
    UPA = DummyUPA()

print(f"PerforatedAI status: {PERFORATED_AI_AVAILABLE}")

In [ ]:
# Helper functions
def count_parameters(model):
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total, trainable

def measure_inference_speed(model, device, img_size=640, runs=50):
    model.eval()
    dummy_input = torch.randn(1, 3, img_size, img_size).to(device)
    
    # Warmup
    for _ in range(5):
        with torch.no_grad():
            _ = model(dummy_input)
    
    if torch.cuda.is_available():
        torch.cuda.synchronize()
    
    start = time.perf_counter()
    for _ in range(runs):
        with torch.no_grad():
            _ = model(dummy_input)
    
    if torch.cuda.is_available():
        torch.cuda.synchronize()
    
    return (time.perf_counter() - start) / runs * 1000

print("✅ Helper functions ready")

In [ ]:
# Initialize W&B and load baseline model
wandb.init(
    project="Dendritic-YOLOv8-Clean",
    name="clean-run",
    tags=["l4-gpu", "clean-start"],
    config={"model": "yolov8n", "dataset": "coco128", "epochs": 3}
)

print("🚀 Loading YOLOv8n...")
baseline_model = YOLO("yolov8n.pt")
baseline_model.model = baseline_model.model.to(device)

# Get parameters
total_params, trainable_params = count_parameters(baseline_model.model)
print(f"📊 Parameters: {total_params/1e6:.2f}M total, {trainable_params/1e6:.2f}M trainable")

wandb.log({"baseline_params_M": total_params/1e6})
print("✅ Baseline model ready")

In [ ]:
# Quick baseline training (3 epochs)
print("🚀 Quick baseline training...")

try:
    results = baseline_model.train(
        data="coco128.yaml",
        epochs=3,  # Reduced for speed
        imgsz=640,
        batch=8,  # Conservative
        device=device,
        project="runs/clean",
        name="baseline",
        exist_ok=True,
        verbose=False  # Reduce output
    )
    print("✅ Baseline training complete")
    
    # Quick validation
    val_results = baseline_model.val(data="coco128.yaml", device=device, verbose=False)
    baseline_map50 = float(val_results.box.map50) if val_results.box.map50 else 0.0
    print(f"📊 Baseline mAP50: {baseline_map50:.4f}")
    
    # Measure speed
    baseline_speed = measure_inference_speed(baseline_model.model, device)
    print(f"⚡ Baseline speed: {baseline_speed:.2f}ms")
    
    baseline_metrics = {
        "mAP50": baseline_map50,
        "params_M": total_params/1e6,
        "speed_ms": baseline_speed
    }
    
    wandb.log({f"baseline_{k}": v for k, v in baseline_metrics.items()})
    
except Exception as e:
    print(f"❌ Training failed: {e}")
    baseline_metrics = {"mAP50": 0.0, "params_M": total_params/1e6, "speed_ms": 0.0}

In [ ]:
# Create dendritic model
print("🧠 Setting up dendritic model...")

# Fresh model for dendritic optimization
dendritic_yolo = YOLO("yolov8n.pt")
dendritic_model = dendritic_yolo.model.to(device)

if PERFORATED_AI_AVAILABLE:
    print("   Configuring PerforatedAI...")
    GPA.pc.set_testing_dendrite_capacity(False)
    GPA.pc.set_verbose(True)
    GPA.pc.set_dendrite_update_mode(True)
    
    try:
        # Apply dendritic optimization
        dendritic_model = UPA.initialize_pai(
            dendritic_model,
            doing_pai=True,
            save_name="CleanDendriticYOLO",
            maximizing_score=True
        )
        print("✅ Dendritic optimization applied")
    except Exception as e:
        print(f"⚠️ PerforatedAI optimization failed: {e}")
        print("   Continuing with standard model")
else:
    print("   Using standard model (PerforatedAI not available)")

# Update YOLO wrapper
dendritic_yolo.model = dendritic_model

# Count parameters
dendritic_total, dendritic_trainable = count_parameters(dendritic_model)
print(f"📊 Dendritic parameters: {dendritic_total/1e6:.2f}M total")
print(f"📊 Parameter change: {((dendritic_total - total_params)/total_params)*100:+.1f}%")

In [ ]:
# Quick dendritic training
print("🚀 Quick dendritic training...")

try:
    dendritic_results = dendritic_yolo.train(
        data="coco128.yaml",
        epochs=3,  # Reduced for speed
        imgsz=640,
        batch=8,
        device=device,
        project="runs/clean",
        name="dendritic",
        exist_ok=True,
        verbose=False
    )
    print("✅ Dendritic training complete")
    
    # Quick validation
    dendritic_val = dendritic_yolo.val(data="coco128.yaml", device=device, verbose=False)
    dendritic_map50 = float(dendritic_val.box.map50) if dendritic_val.box.map50 else 0.0
    print(f"📊 Dendritic mAP50: {dendritic_map50:.4f}")
    
    # Add validation score to PerforatedAI tracker
    if PERFORATED_AI_AVAILABLE:
        try:
            dendritic_model, restructured, complete = GPA.pai_tracker.add_validation_score(
                dendritic_map50, dendritic_model
            )
            if restructured:
                print("🔄 Model restructured by PerforatedAI")
                dendritic_yolo.model = dendritic_model
            if complete:
                print("✅ PerforatedAI optimization cycle complete")
        except Exception as e:
            print(f"⚠️ PerforatedAI validation scoring failed: {e}")
    
    # Measure speed
    dendritic_speed = measure_inference_speed(dendritic_yolo.model, device)
    print(f"⚡ Dendritic speed: {dendritic_speed:.2f}ms")
    
    dendritic_metrics = {
        "mAP50": dendritic_map50,
        "params_M": dendritic_total/1e6,
        "speed_ms": dendritic_speed
    }
    
    wandb.log({f"dendritic_{k}": v for k, v in dendritic_metrics.items()})
    
except Exception as e:
    print(f"❌ Dendritic training failed: {e}")
    dendritic_metrics = {"mAP50": 0.0, "params_M": dendritic_total/1e6, "speed_ms": 0.0}

In [ ]:
# Results comparison and visualization
print("📊 Generating results...")

# Calculate improvements
improvements = {}
if baseline_metrics['params_M'] > 0:
    improvements['param_reduction_pct'] = ((baseline_metrics['params_M'] - dendritic_metrics['params_M']) / baseline_metrics['params_M']) * 100

improvements['mAP50_change'] = dendritic_metrics['mAP50'] - baseline_metrics['mAP50']

if baseline_metrics['speed_ms'] > 0:
    improvements['speed_change_pct'] = ((baseline_metrics['speed_ms'] - dendritic_metrics['speed_ms']) / baseline_metrics['speed_ms']) * 100

# Create comparison chart
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# mAP comparison
map_vals = [baseline_metrics['mAP50'], dendritic_metrics['mAP50']]
axes[0].bar(['Baseline', 'Dendritic'], map_vals, color=['steelblue', 'coral'])
axes[0].set_ylabel('mAP50')
axes[0].set_title('mAP50 Comparison')

# Parameters comparison
param_vals = [baseline_metrics['params_M'], dendritic_metrics['params_M']]
axes[1].bar(['Baseline', 'Dendritic'], param_vals, color=['steelblue', 'coral'])
axes[1].set_ylabel('Parameters (M)')
axes[1].set_title('Model Size')

# Speed comparison
speed_vals = [baseline_metrics['speed_ms'], dendritic_metrics['speed_ms']]
axes[2].bar(['Baseline', 'Dendritic'], speed_vals, color=['steelblue', 'coral'])
axes[2].set_ylabel('Inference Time (ms)')
axes[2].set_title('Inference Speed')

plt.tight_layout()
plt.savefig('clean_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

# Summary
print("\n" + "="*60)
print("🏆 CLEAN DENDRITIC YOLOV8 RESULTS")
print("="*60)
print(f"📊 Baseline:  mAP50={baseline_metrics['mAP50']:.4f}, Params={baseline_metrics['params_M']:.2f}M, Speed={baseline_metrics['speed_ms']:.1f}ms")
print(f"📊 Dendritic: mAP50={dendritic_metrics['mAP50']:.4f}, Params={dendritic_metrics['params_M']:.2f}M, Speed={dendritic_metrics['speed_ms']:.1f}ms")
print(f"\n📈 Improvements:")
for key, value in improvements.items():
    print(f"   {key}: {value:+.2f}{'%' if 'pct' in key else ''}")
print(f"\n🔧 PerforatedAI: {'✅ Active' if PERFORATED_AI_AVAILABLE else '❌ Not Available'}")
print("="*60)

wandb.finish()

In [ ]:
# Save final results
results = {
    "project": "Dendritic YOLOv8 - Clean Implementation",
    "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
    "environment": {
        "device": device,
        "gpu_name": torch.cuda.get_device_name(0) if torch.cuda.is_available() else "none",
        "pytorch_version": torch.__version__,
        "perforated_ai_available": PERFORATED_AI_AVAILABLE
    },
    "baseline": baseline_metrics,
    "dendritic": dendritic_metrics,
    "improvements": improvements,
    "success": True
}

with open('clean_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("✅ Results saved to 'clean_results.json'")
print("✅ Chart saved to 'clean_comparison.png'")
print("\n🎯 Clean execution complete!")
print("📁 Download files: clean_results.json, clean_comparison.png")